In [1]:
import os
import re
import json
import pandas as pd

In [2]:
texts = []
texts_dir_path = './data_zhihu/depression_patients/'
for i in range(1, 16):
    with open(os.path.join(texts_dir_path, '{:02}.txt'.format(i)), 'r') as f:
        texts.append(f.read())

In [3]:
vocabs = {}
vocabs_dir_path = './vocab'
topns = [25, 50, 75, 100]
thresholds = [0.7, 0.75, 0.8, 0.85]
vocab_names = ['new_vocab_{}_{}'.format(topn, thresh) for topn in topns for thresh in thresholds]
vocab_names.append('old_vocab')
for vocab_name in vocab_names:
    with open(os.path.join(vocabs_dir_path, vocab_name+'.dic'), 'r', encoding='utf-8') as f:
        vocab = {}
        for line in f.readlines()[83:]:
            x = line.strip().split('\t')
            vocab[x[0]] = x[1:]
        vocabs[vocab_name] = vocab

In [4]:
word_types = [
1, # function (Function Words)
	2, # pronoun (Pronouns)
		3, # ppron (Personal Pronouns)
			4, # i (I)
			5, # we (We)
			6, # you (You)
			7, # shehe (SheHe)
			8, # they (They)
			19, # youpl
		9, # ipron (Impersonal Pronouns)
	11, # prep (Prepositions)
	12, # auxverb (Auxiliary Verbs)
	13, # adverb (Adverbs)
	14, # conj (Conjunctions)
	15, # negate (Negations)
	17, # quanunit
	18, # prepend
	131, # specart
	132, # tensem
		90, # focuspast (Past Focus)
		91, # focuspresent (Present Focus)
		92, # focusfuture (Future Focus)
		133, # progm
	140, # particle
		141, # modal_pa
		142, # general_pa
# othergram (Other Grammar)
	22, # compare (Comparisons)
	23, # interrog (Interrogatives)
	24, # number (Numbers)
	25, # quant (Quantifiers)
30, # affect (Affect)
	31, # posemo (Positive Emotions)
	32, # negemo (Negative Emotions)
		33, # anx (Anx)
		34, # anger (Anger)
		35, # sad (Sad)
40, # social (Social)
	41, # family (Family)
	42, # friend (Friends)
	43, # female (Female)
	44, # male (Male)
50, # cogproc (Cognitive Processes)
	51, # insight (Insight)
	52, # cause (Causal)
	53, # discrep (Discrepancies)
	54, # tentat (Tentative)
	55, # certain (Certainty)
	56, # differ (Differentiation)
60, # percept (Perceptual Processes)
	61, # see (See)
	62, # hear (Hear)
	63, # feel (Feel)
70, # bio (Biological Processes)
	71, # body (Body)
	72, # health (Health)
	73, # sexual (Sexual)
	74, # ingest (Ingest)
80, # drives (Drives)
	81, # affiliation (Affiliation)
	82, # achieve (Achievement)
	83, # power (Power)
	84, # reward (Reward)
	85, # risk (Risk)
100, # relativ (Relativity)
	101, # motion (Motion)
	102, # space (Space)
	103, # time (Time)
# persconc (Personal Concerns)
	110, # work (Work)
	111, # leisure (Leisure)
	112, # home (Home)
	113, # money (Money)
	114, # relig (Religion)
	115, # death (Death)
120, # informal (Informal Language)
	121, # swear (Swear)
	122, # netspeak (Netspeak)
	123, # assent (Assent)
	124, # nonflu (Nonfluencies)
	125, # filler (Filler Words)
]

In [5]:
word_counts = {}
word_type_counts = {}
for vocab_name in vocab_names:
    print('counting', vocab_name)
    vocab = vocabs[vocab_name]
    word_counts[vocab_name] = {}
    word_type_counts[vocab_name] = {}
    for i in range(1, 16):
        text_name = 'text_{:02}'.format(i)
        print('  {:02}'.format(i), end='')
        word_counts[vocab_name][text_name] = {}
        word_type_counts[vocab_name][text_name] = {}
        for word_type in word_types:
            word_type_counts[vocab_name][text_name][word_type] = 0
        text = texts[i-1]
        for word in vocab:
            word_to_check = word
            word_to_check = word_to_check.replace('*', '.')
            word_to_check = word_to_check.replace('(', '\(')
            word_to_check = word_to_check.replace(')', '\)')
            matches = re.findall(word_to_check, text)
            if len(matches) > 0:
                word_counts[vocab_name][text_name][word] = len(matches)
                for word_type in vocab[word]:
                    word_type_counts[vocab_name][text_name][int(word_type)] += len(matches)
    print()

counting new_vocab_25_0.7
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_25_0.75
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_25_0.8
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_25_0.85
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_50_0.7
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_50_0.75
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_50_0.8
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_50_0.85
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_75_0.7
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_75_0.75
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_75_0.8
  01  02  03  04  05  06  07  08  09  10  11  12  13  14  15
counting new_vocab_75_0.85
  01  02  0

In [6]:
with open('./word_counts/word_counts.json', 'w') as f:
    f.write(json.dumps(word_counts, ensure_ascii=False, indent=2))
with open('./word_counts/word_type_counts.json', 'w') as f:
    f.write(json.dumps(word_type_counts, ensure_ascii=False, indent=2))

In [7]:
for vocab_name in vocab_names:
    df = pd.DataFrame(word_type_counts[vocab_name]).T
    df.to_csv('./word_counts/{}.csv'.format(vocab_name))